In [56]:
# Example Running of Khmer Fingerspelling Detection
# Update and Test by Ye Kyaw Thu, Affiliate Professor, CADT, Cambodia
# Demo for my two ITC internship students
# Reference: https://github.com/nicknochnack/ActionDetectionforSignLanguage
# Last updated: 30 Jan 2023

In [57]:
# After I prepared data with OpenCV on local Linux notebook, 
# I run this notebook on server

# 6. Preprocess Data and Create Labels and Features

In [58]:
import numpy as np
import os
from matplotlib import pyplot as plt
import time

In [59]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [60]:
!pwd

/home/yekyaw.thu/exp/fs-detection


In [61]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('fs_data') 

# Actions that we try to detect
#actions = np.array(['ក', 'ខ', 'គ', 'ឃ', 'ង'])
actions = np.array(['ka', 'kha', 'ko', 'kho', 'ngo'])

# Videos are going to be 30 frames in length
sequence_length = 30

In [62]:
label_map = {label:num for num, label in enumerate(actions)}

In [63]:
label_map

{'ka': 0, 'kha': 1, 'ko': 2, 'kho': 3, 'ngo': 4}

In [64]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [65]:
# 180 vs 150 ...
np.array(sequences).shape

(150, 30, 1662)

In [66]:
np.array(labels).shape

(150,)

In [67]:
X = np.array(sequences)

In [68]:
X.shape

(150, 30, 1662)

In [69]:
y = to_categorical(labels).astype(int)

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [71]:
y_test.shape

(8, 5)

# 7. Build and Train LSTM Neural Network


In [72]:
!nvidia-smi

Mon Jan 30 17:32:56 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:0A:00.0 Off |                  N/A |
| 23%   49C    P8    21W / 300W |  10184MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:42:00.0 Off |                  N/A |
|  0%   

In [73]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [74]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [75]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [76]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [77]:
model.fit(X_train, y_train, epochs=1000, callbacks=[tb_callback])

Epoch 1/1000
5/5 [==============================] - 4s 105ms/step - loss: 15.3507 - categorical_accuracy: 0.1901
Epoch 2/1000
5/5 [==============================] - 1s 109ms/step - loss: 86.6774 - categorical_accuracy: 0.1620
Epoch 3/1000
5/5 [==============================] - 1s 100ms/step - loss: 45.6215 - categorical_accuracy: 0.1972
Epoch 4/1000
5/5 [==============================] - 1s 105ms/step - loss: 29.8255 - categorical_accuracy: 0.1620
Epoch 5/1000
5/5 [==============================] - 1s 106ms/step - loss: 37.1070 - categorical_accuracy: 0.1761
Epoch 6/1000
5/5 [==============================] - 1s 106ms/step - loss: 58.6515 - categorical_accuracy: 0.1761
Epoch 7/1000
5/5 [==============================] - 1s 104ms/step - loss: 193.7306 - categorical_accuracy: 0.1831
Epoch 8/1000
5/5 [==============================] - 1s 105ms/step - loss: 302.4875 - categorical_accuracy: 0.2324
Epoch 9/1000
5/5 [==============================] - 1s 107ms/step - loss: 239.0022 - categoric

5/5 [==============================] - 1s 105ms/step - loss: 2.2749 - categorical_accuracy: 0.2465
Epoch 146/1000
5/5 [==============================] - 1s 108ms/step - loss: 2.0568 - categorical_accuracy: 0.2254
Epoch 147/1000
5/5 [==============================] - 1s 104ms/step - loss: 2.0492 - categorical_accuracy: 0.1620
Epoch 148/1000
5/5 [==============================] - 1s 103ms/step - loss: 2.1589 - categorical_accuracy: 0.2183
Epoch 149/1000
5/5 [==============================] - 1s 104ms/step - loss: 1.9966 - categorical_accuracy: 0.1972
Epoch 150/1000
5/5 [==============================] - 1s 106ms/step - loss: 2.8213 - categorical_accuracy: 0.1690
Epoch 151/1000
5/5 [==============================] - 1s 102ms/step - loss: 2.3320 - categorical_accuracy: 0.1972
Epoch 152/1000
5/5 [==============================] - 1s 105ms/step - loss: 2.0124 - categorical_accuracy: 0.2254
Epoch 153/1000
5/5 [==============================] - 1s 104ms/step - loss: 2.2641 - categorical_accura

Epoch 217/1000
5/5 [==============================] - 1s 109ms/step - loss: 1.7120 - categorical_accuracy: 0.2042
Epoch 218/1000
5/5 [==============================] - 1s 105ms/step - loss: 1.8286 - categorical_accuracy: 0.2324
Epoch 219/1000
5/5 [==============================] - 1s 104ms/step - loss: 1.7952 - categorical_accuracy: 0.2042
Epoch 220/1000
5/5 [==============================] - 1s 106ms/step - loss: 1.8408 - categorical_accuracy: 0.1831
Epoch 221/1000
5/5 [==============================] - 1s 104ms/step - loss: 1.8059 - categorical_accuracy: 0.1901
Epoch 222/1000
5/5 [==============================] - 1s 104ms/step - loss: 2.0372 - categorical_accuracy: 0.2113
Epoch 223/1000
5/5 [==============================] - 1s 102ms/step - loss: 2.0680 - categorical_accuracy: 0.2606
Epoch 224/1000
5/5 [==============================] - 1s 107ms/step - loss: 1.9250 - categorical_accuracy: 0.1831
Epoch 225/1000
5/5 [==============================] - 1s 107ms/step - loss: 1.9743 - cat

5/5 [==============================] - 1s 107ms/step - loss: 1.9364 - categorical_accuracy: 0.2887
Epoch 361/1000
5/5 [==============================] - 1s 105ms/step - loss: 2.0605 - categorical_accuracy: 0.1761
Epoch 362/1000
5/5 [==============================] - 1s 106ms/step - loss: 2.4120 - categorical_accuracy: 0.1831
Epoch 363/1000
5/5 [==============================] - 1s 105ms/step - loss: 1.7361 - categorical_accuracy: 0.2465
Epoch 364/1000
5/5 [==============================] - 1s 105ms/step - loss: 1.7620 - categorical_accuracy: 0.2042
Epoch 365/1000
5/5 [==============================] - 1s 106ms/step - loss: 1.6399 - categorical_accuracy: 0.2465
Epoch 366/1000
5/5 [==============================] - 1s 104ms/step - loss: 1.8198 - categorical_accuracy: 0.2042
Epoch 367/1000
5/5 [==============================] - 1s 111ms/step - loss: 1.9707 - categorical_accuracy: 0.2324
Epoch 368/1000
5/5 [==============================] - 1s 107ms/step - loss: 2.0159 - categorical_accura

5/5 [==============================] - 1s 105ms/step - loss: 2.2416 - categorical_accuracy: 0.2324
Epoch 433/1000
5/5 [==============================] - 1s 103ms/step - loss: 2.8893 - categorical_accuracy: 0.1901
Epoch 434/1000
5/5 [==============================] - 1s 105ms/step - loss: 2.8602 - categorical_accuracy: 0.2042
Epoch 435/1000
5/5 [==============================] - 1s 105ms/step - loss: 2.2969 - categorical_accuracy: 0.2535
Epoch 436/1000
5/5 [==============================] - 1s 107ms/step - loss: 2.2067 - categorical_accuracy: 0.2183
Epoch 437/1000
5/5 [==============================] - 1s 101ms/step - loss: 1.9295 - categorical_accuracy: 0.2254
Epoch 438/1000
5/5 [==============================] - 1s 102ms/step - loss: 2.2753 - categorical_accuracy: 0.2254
Epoch 439/1000
5/5 [==============================] - 1s 103ms/step - loss: 1.9894 - categorical_accuracy: 0.2324
Epoch 440/1000
5/5 [==============================] - 1s 107ms/step - loss: 2.0409 - categorical_accura

Epoch 504/1000
5/5 [==============================] - 1s 109ms/step - loss: 1.7352 - categorical_accuracy: 0.2606
Epoch 505/1000
5/5 [==============================] - 1s 104ms/step - loss: 1.6945 - categorical_accuracy: 0.2254
Epoch 506/1000
5/5 [==============================] - 1s 103ms/step - loss: 1.7388 - categorical_accuracy: 0.2394
Epoch 507/1000
5/5 [==============================] - 1s 106ms/step - loss: 1.7769 - categorical_accuracy: 0.2324
Epoch 508/1000
5/5 [==============================] - 1s 103ms/step - loss: 1.7861 - categorical_accuracy: 0.2394
Epoch 509/1000
5/5 [==============================] - 1s 105ms/step - loss: 1.8698 - categorical_accuracy: 0.1972
Epoch 510/1000
5/5 [==============================] - 1s 101ms/step - loss: 1.9596 - categorical_accuracy: 0.2887
Epoch 511/1000
5/5 [==============================] - 1s 101ms/step - loss: 2.1361 - categorical_accuracy: 0.2254
Epoch 512/1000
5/5 [==============================] - 1s 109ms/step - loss: 2.0437 - cat

5/5 [==============================] - 1s 103ms/step - loss: 1.8807 - categorical_accuracy: 0.2535
Epoch 648/1000
5/5 [==============================] - 1s 107ms/step - loss: 1.8399 - categorical_accuracy: 0.2606
Epoch 649/1000
5/5 [==============================] - 1s 101ms/step - loss: 1.7583 - categorical_accuracy: 0.2183
Epoch 650/1000
5/5 [==============================] - 1s 106ms/step - loss: 1.8783 - categorical_accuracy: 0.2746
Epoch 651/1000
5/5 [==============================] - 1s 105ms/step - loss: 1.7658 - categorical_accuracy: 0.2394
Epoch 652/1000
5/5 [==============================] - 1s 109ms/step - loss: 1.7966 - categorical_accuracy: 0.2113
Epoch 653/1000
5/5 [==============================] - 1s 105ms/step - loss: 1.7561 - categorical_accuracy: 0.2394
Epoch 654/1000
5/5 [==============================] - 1s 108ms/step - loss: 1.8022 - categorical_accuracy: 0.2324
Epoch 655/1000
5/5 [==============================] - 1s 106ms/step - loss: 1.8446 - categorical_accura

Epoch 719/1000
5/5 [==============================] - 1s 104ms/step - loss: 1.7667 - categorical_accuracy: 0.1761
Epoch 720/1000
5/5 [==============================] - 1s 105ms/step - loss: 1.8137 - categorical_accuracy: 0.2183
Epoch 721/1000
5/5 [==============================] - 1s 105ms/step - loss: 1.6603 - categorical_accuracy: 0.2676
Epoch 722/1000
5/5 [==============================] - 1s 108ms/step - loss: 1.6776 - categorical_accuracy: 0.2394
Epoch 723/1000
5/5 [==============================] - 1s 106ms/step - loss: 1.6326 - categorical_accuracy: 0.2535
Epoch 724/1000
5/5 [==============================] - 1s 106ms/step - loss: 1.6076 - categorical_accuracy: 0.3099
Epoch 725/1000
5/5 [==============================] - 1s 108ms/step - loss: 1.8158 - categorical_accuracy: 0.1972
Epoch 726/1000
5/5 [==============================] - 1s 106ms/step - loss: 1.7781 - categorical_accuracy: 0.2606
Epoch 727/1000
5/5 [==============================] - 1s 104ms/step - loss: 1.7819 - cat

5/5 [==============================] - 1s 101ms/step - loss: 1.7743 - categorical_accuracy: 0.2394
Epoch 863/1000
5/5 [==============================] - 1s 102ms/step - loss: 1.8160 - categorical_accuracy: 0.2183
Epoch 864/1000
5/5 [==============================] - 1s 105ms/step - loss: 1.8673 - categorical_accuracy: 0.1479
Epoch 865/1000
5/5 [==============================] - 1s 106ms/step - loss: 1.7842 - categorical_accuracy: 0.1761
Epoch 866/1000
5/5 [==============================] - 1s 104ms/step - loss: 1.7627 - categorical_accuracy: 0.2254
Epoch 867/1000
5/5 [==============================] - 1s 108ms/step - loss: 1.6225 - categorical_accuracy: 0.2254
Epoch 868/1000
5/5 [==============================] - 1s 104ms/step - loss: 1.8012 - categorical_accuracy: 0.1620
Epoch 869/1000
5/5 [==============================] - 1s 105ms/step - loss: 1.8105 - categorical_accuracy: 0.1901
Epoch 870/1000
5/5 [==============================] - 1s 106ms/step - loss: 1.7162 - categorical_accura

Epoch 934/1000
5/5 [==============================] - 1s 106ms/step - loss: 1.6607 - categorical_accuracy: 0.2606
Epoch 935/1000
5/5 [==============================] - 1s 103ms/step - loss: 1.7125 - categorical_accuracy: 0.2113
Epoch 936/1000
5/5 [==============================] - 1s 108ms/step - loss: 1.6761 - categorical_accuracy: 0.2887
Epoch 937/1000
5/5 [==============================] - 1s 101ms/step - loss: 1.6959 - categorical_accuracy: 0.2324
Epoch 938/1000
5/5 [==============================] - 1s 104ms/step - loss: 1.8005 - categorical_accuracy: 0.1761
Epoch 939/1000
5/5 [==============================] - 1s 104ms/step - loss: 1.7075 - categorical_accuracy: 0.2183
Epoch 940/1000
5/5 [==============================] - 1s 106ms/step - loss: 1.7339 - categorical_accuracy: 0.2324
Epoch 941/1000
5/5 [==============================] - 1s 108ms/step - loss: 1.6381 - categorical_accuracy: 0.2324
Epoch 942/1000
5/5 [==============================] - 1s 109ms/step - loss: 1.6972 - cat

In [79]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 64)            442112    
                                                                 
 lstm_4 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 5)                 165       
                                                                 
Total params: 596,741
Trainable params: 596,741
Non-tr

# 8. Make Predictions

In [80]:
res = model.predict(X_test)

1/1 [==============================] - 0s 279ms/step


In [86]:
actions[np.argmax(res[3])]

'ka'

In [87]:
actions[np.argmax(y_test[3])]

'ka'

# 9. Save Weights

In [88]:
model.save('kh-fs-5-consonant-1000epoch.h5')

In [35]:
# delete the current model
del model

In [38]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))
model.load_weights('kh-fs-5-consonant-2000epoch.h5')

# 10. Evaluation using Confusion Matrix and Accuracy


In [89]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [90]:
yhat = model.predict(X_test)

1/1 [==============================] - 0s 36ms/step


In [91]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [92]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[5, 1],
        [1, 1]],

       [[3, 4],
        [0, 1]],

       [[7, 0],
        [1, 0]],

       [[4, 1],
        [3, 0]],

       [[7, 0],
        [1, 0]]])

In [93]:
accuracy_score(ytrue, yhat)

0.25